### Testing on the looping procedure

In [11]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *

In [12]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *



class Calculator():
    def __init__(self):
        
        # COMMENT THIS IF FLASK CONNECTION IS WORKING
        self.cpi = 0.12
        self.markup_percentage = 0.001
        self.maintenance_ratio = 0.08
        self.warranty_rate = 0.05
        self.insurance_rate = 0.015
        self.travel_labor_cost = 300
        self.business_con_rate = 0.02
        self.monthly_interest_rate = self.cpi/12
        # self.monthly_interest_rate = ((1+self.cpi)**(1/12))-1
        
        # UNCOMMENT THIS IF FLASK CONNECTION IS WORKING
        response = requests.get("http://127.0.0.1:5000/get_parameters")
        if response.status_code == 200:
            params = response.json()
            self.cpi = params['cpi']
            self.markup_percentage = params['markup_percentage']
            self.maintenance_ratio = params['maintenance_ratio']
            self.warranty_rate = params['warranty_rate']
            self.insurance_rate = params['insurance_rate']
            self.travel_labor_cost = params['travel_labor_cost']
            self.business_con_rate = params['business_con_rate']
        else:
            st.error("Failed to load parameters!")
            
        self.name = None
        self.monthlyPayment = None
        self.totalPayment = None
        self.invoice = None
                 


    def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        principal = markup_price
        # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
 
        # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        travel_labor_cost = self.travel_labor_cost * LoanTerm
        business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
        
        
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        total_payment = total_added_value_services + principal
        
        terminal_value = markup_price * terminal_rate 
        
        
        monthly_interest_rate = self.monthly_interest_rate
        monthly_added_value_payment = total_added_value_services / (LoanTerm*12)
        monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -principal) + monthly_added_value_payment

        self.monthlyPayment = monthlyPayment
        
        results = {
            "principal": principal,
            "total_payment": total_payment, # principal + total_added_value_services
            'total_added_value': total_added_value_services,
            "monthlyPayment": monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "business_con_fee": business_con_fee,
            "terminal_value": terminal_value,
            'travel_labor_cost': travel_labor_cost
        }

        return results
    
    def getLoanTerm(self, EquipmentPrice, monthlyPayment, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        # Calculating the markup price (principal)
        markup_price = EquipmentPrice  # as it has been marked up in the input form
        principal = markup_price
        
        # Calculating the maintenance fee
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
        
        # Calculating the warranty fee
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        
        # Calculating the insurance fee
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        
        # Travel labor cost
        travel_labor_cost = self.travel_labor_cost
        
        # Calculating the terminal value
        terminal_value = markup_price * terminal_rate
        
        # Calculating the business continuity fee
        business_con_fee = markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if bussiness_con == 'Yes' else 0

        # Total added value services (maintenance, insurance, etc.), NOT affected by interest
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        
        # Define monthly interest rate
        monthly_interest_rate = self.monthly_interest_rate
        
        # First, calculate the LoanTerm in months based on the principal only
        LoanTerm_months = npf.nper(monthly_interest_rate, -monthlyPayment, principal, 0)
        
        # Round up to the nearest whole number of months
        LoanTerm_months_rounded = math.ceil(LoanTerm_months)
        
        # Now, calculate the portion of the monthly payment that covers the added value services
        monthly_added_value_payment = total_added_value_services / LoanTerm_months_rounded
        
        # Adjust the monthly payment for the principal by subtracting the portion for added value services
        adjusted_monthlyPayment = monthlyPayment - monthly_added_value_payment
        
        # Calculate the remaining balance after making LoanTerm_months_rounded - 1 payments
        remaining_balance = npf.fv(monthly_interest_rate, LoanTerm_months_rounded - 1, -adjusted_monthlyPayment, principal)
        
        # Calculate the final payment to settle the remaining balance
        final_payment = remaining_balance * (1 + monthly_interest_rate)
        
        last_monthlyPayment = -final_payment + monthly_added_value_payment  # Adjust last payment to include the added value payment
        
        # Calculate the LoanTerm in years
        LoanTerm_years = LoanTerm_months_rounded / 12

        # Save the loan term and the last month's payment
        self.LoanTerm = LoanTerm_years
        self.last_monthlyPayment = last_monthlyPayment

        results = {
            "principal": principal,  # This is now just the principal
            "total_payment": principal + total_added_value_services,  # Total payment over the loan term
            'total_added_value': total_added_value_services,  # Total added value services
            "LoanTerm_years": LoanTerm_years,
            "LoanTerm_months": LoanTerm_months_rounded,
            "monthly_added_value_payment": monthly_added_value_payment,  # Fixed payment for added value services
            "last_monthlyPayment": last_monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "terminal_value": terminal_value,
            "business_con_fee": business_con_fee,
            "travel_labor_cost": travel_labor_cost
        }

        return results


    def getInvoice(self, EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con ): 
        invoice = self.getMonthlyPayment(EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con)['monthlyPayment'] * LoanTerm * 12
        return invoice

    def setName(self, name):
        self.name = name

    def displayResult(self, result):
        st.subheader(f"{self.name}")
        st.write(f"{result}")

In [13]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 7500

# Loop to generate data for 200 entries
for i in range(120):
    equipment_price = a
    loan_term = get_pricing_details(equipment_price)['Loan Term']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [14]:
df

,EquipmentPrice,LoanTerm
0,7500,3.0
1,10000,3.0
2,12500,3.0
3,15000,3.0
4,17500,3.0
...,...,...
115,295000,10.0
116,297500,10.0
117,300000,10.0
118,302500,10.0


In [15]:
clinic = Calculator()

list = []

for i in range(120):

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])
    
    

    

In [16]:
df['EquipmentPrice'].values

array([  7500,  10000,  12500,  15000,  17500,  20000,  22500,  25000,
        27500,  30000,  32500,  35000,  37500,  40000,  42500,  45000,
        47500,  50000,  52500,  55000,  57500,  60000,  62500,  65000,
        67500,  70000,  72500,  75000,  77500,  80000,  82500,  85000,
        87500,  90000,  92500,  95000,  97500, 100000, 102500, 105000,
       107500, 110000, 112500, 115000, 117500, 120000, 122500, 125000,
       127500, 130000, 132500, 135000, 137500, 140000, 142500, 145000,
       147500, 150000, 152500, 155000, 157500, 160000, 162500, 165000,
       167500, 170000, 172500, 175000, 177500, 180000, 182500, 185000,
       187500, 190000, 192500, 195000, 197500, 200000, 202500, 205000,
       207500, 210000, 212500, 215000, 217500, 220000, 222500, 225000,
       227500, 230000, 232500, 235000, 237500, 240000, 242500, 245000,
       247500, 250000, 252500, 255000, 257500, 260000, 262500, 265000,
       267500, 270000, 272500, 275000, 277500, 280000, 282500, 285000,
      

In [17]:
list

[290.7739902630504,
 379.3653203507339,
 467.95665043841734,
 556.5479805261008,
 645.1393106137843,
 733.7306407014678,
 555.5000729102896,
 614.4445254558774,
 673.388978001465,
 732.3334305470529,
 791.2778830926406,
 850.2223356382284,
 909.166788183816,
 968.1112407294039,
 1027.0556932749917,
 1086.0001458205793,
 1144.944598366167,
 1203.8890509117548,
 1001.7684718463751,
 1048.281256220012,
 1094.7940405936488,
 1141.3068249672858,
 1187.8196093409229,
 1234.3323937145597,
 1280.8451780881965,
 1327.3579624618335,
 1373.8707468354703,
 1420.383531209107,
 1466.8963155827444,
 1513.4090999563812,
 1559.9218843300182,
 1606.434668703655,
 1652.9474530772918,
 1699.4602374509288,
 1745.9730218245656,
 1792.4858061982027,
 1838.9985905718395,
 1885.5113749454763,
 1563.9105544598533,
 1601.444958227167,
 1638.9793619944805,
 1676.5137657617938,
 1714.0481695291073,
 1751.5825732964208,
 1789.1169770637343,
 1826.6513808310478,
 1864.1857845983614,
 1901.7201883656746,
 1939.254592

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')
plt.show()

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

plt.axvline(x=df['EquipmentPrice'].iloc[5], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[10], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[20], color='red', linestyle='--', label='Loan Term = 2 yrs')

plt.xlabel('Equipment Price')
plt.ylabel('Monthly Payment')
# plt.legend()
plt.savefig('test.png')

### Generate Random DF

In [20]:
import random

In [21]:
import pandas as pd

# Define the data as a dictionary with numerical representations
data = {
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [5000, 20000, 50000, 10000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Loan Term': [3, 5, 7, 7, 7],
    'Markup Price': [5.0, 4.0, 2.0, 1.0, 1.0],
    'Warranty': [10.0, 5.0, 3.0, 1.0, 1.0],
    'Maintenance': [15.0, 10.0, 8.0, 5.0, 5.0],
    'Insurance': [10.0, 7.0, 5.0, 1.5, 1.5],
    'Business Con': [10.0, 5.0, 3.0, 2.0, 2.0],
    # 'Terminal Value': [5.0, 5.0, 3.0, 1.0, 1.0]
}

# Create a DataFrame
df_numeric = pd.DataFrame(data)



In [22]:
def generate_random_data_with_ranges(df, column_ranges):
    """
    Generate random integer data for the DataFrame based on specified column ranges.
    Each column must select its values from a predefined list in the column_ranges dictionary.
    Columns with a specified list will follow the sort rule:
    - Non-decreasing for 'Loan Term' (formerly 'Free Warranty Years')
    - Non-increasing for all other columns.

    Parameters:
        df (pd.DataFrame): The original DataFrame containing the data.
        column_ranges (dict): A dictionary specifying the allowed values for each column.
    
    Returns:
        pd.DataFrame: DataFrame with randomly generated integer values ordered based on rules.
    """
    
    # Create a copy of the dataframe to avoid modifying the original
    df_random = df.copy()

    # Iterate through each column and generate random values from the list in column_ranges
    for column, value_range in column_ranges.items():
        # Randomly select values from the provided range for each column
        random_values = [random.choice(value_range) for _ in range(5)]
        
        # Apply sorting rules based on the column
        if column == 'Loan Term':
            # Sort 'Loan Term' in non-decreasing order (ascending)
            random_values.sort()
        else:
            # Sort other columns in non-increasing order (descending)
            random_values.sort(reverse=True)
        
        # Assign the sorted values back to the DataFrame
        df_random[column] = random_values

    return df_random




In [23]:
# Define a dictionary with column names as keys and allowed values as lists
column_ranges = {
    'Loan Term': [2, 3, 5, 7],
    'Markup Price': [1, 3, 5, 7, 9],
    'Warranty': [2, 4, 6, 8, 10],
    'Maintenance': [5, 10, 12, 15],
    'Insurance': [1, 2, 4, 5, 7, 10],
    'Business Con': [1, 3, 5, 7, 10],
    # 'Terminal Value': [1, 2, 3, 4, 5]
    }


df_random = generate_random_data_with_ranges(df_numeric, column_ranges)

In [24]:
df_random

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,7,8,12,5,7
1,2,20000,50000,2,5,8,12,2,5
2,3,50000,100000,3,3,6,10,2,5
3,4,10000,200000,3,1,2,10,1,3
4,5,200000,1000000,5,1,2,5,1,3


In [25]:
def get_pricing_details_from_df(equipment_price, df):
    """
    Function to get pricing details based on the equipment price from a given DataFrame.
    
    Parameters:
        equipment_price (float): The price of the equipment.
        df (pd.DataFrame): The DataFrame containing pricing data.
        
    Returns:
        dict: A dictionary containing the pricing details for the corresponding class.
    """
    
    # Find the row where the equipment price falls between the bottom and upper price
    row = df[(df['Bottom Price'] < equipment_price) & (df['Upper Price'] >= equipment_price)]
    
    # If no class matches, return None
    if row.empty:
        return None
    
    # Convert the row to a dictionary and return
    pricing_details = row.iloc[0].to_dict()
    return pricing_details

In [26]:
get_pricing_details_from_df(30000,df_random)

{'Class': 2,
 'Bottom Price': 20000,
 'Upper Price': 50000,
 'Loan Term': 2,
 'Markup Price': 5,
 'Warranty': 8,
 'Maintenance': 12,
 'Insurance': 2,
 'Business Con': 5}

### Running on random df generation

In [27]:
df_random

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,7,8,12,5,7
1,2,20000,50000,2,5,8,12,2,5
2,3,50000,100000,3,3,6,10,2,5
3,4,10000,200000,3,1,2,10,1,3
4,5,200000,1000000,5,1,2,5,1,3


In [28]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 7500

# Loop to generate data for 200 entries
for i in range(100):
    equipment_price = a
    loan_term = get_pricing_details_from_df(equipment_price,df_random)['Loan Term']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [29]:
clinic = Calculator()

list = []

for i in range(100):
    
    

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])

In [30]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

plt.axvline(x=df['EquipmentPrice'].iloc[3], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[9], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[19], color='red', linestyle='--', label='Loan Term = 2 yrs')

plt.xlabel('Equipment Price')
plt.ylabel('Monthly Payment')
# plt.legend()
plt.savefig('test.png')

### Build as function

In [62]:
import pandas as pd
from parameter import *
import seaborn as sns
import matplotlib.pyplot as plt
from parameter import *

In [63]:
# Define a dictionary with column names as keys and allowed values as lists
column_ranges = {
    'Loan Term': [2, 3, 5, 7],
    'Markup Price': [1, 3, 5, 7, 9],
    'Warranty': [2, 4, 6, 8, 10],
    'Maintenance': [5, 7, 10],
    'Insurance': [1, 2, 3, 5, 7, 10],
    'Business Con': [1, 3, 5, 7],
}

In [64]:
def run_random_sim():

    # randomize parameter configuration
    
    
    df_random = generate_random_data_with_ranges(df_numeric, column_ranges)

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    a = 7500

    # Loop to generate data for 200 entries
    for i in range(120):
        equipment_price = a
        response = get_pricing_details_from_df(equipment_price,df_random)
        loan_term = response['Loan Term']
        data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
        
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con'],
            # 'Terminal Value': response['Terminal Value']
            
        })
        
        a += 2500

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    clinic = Calculator()

    list = []
    total_payment = []
    finance_product = []

    for i in range(120):
        
        clinic.markup_percentage = params[i]['Markup Price'] / 100
        clinic.warranty_rate = params[i]['Warranty'] / 100
        clinic.maintenance_ratio = params[i]['Maintenance'] / 100
        clinic.insurance_rate = params[i]['Insurance'] / 100
        clinic.business_con_rate = params[i]['Business Con'] / 100
        warranty = df['LoanTerm'][i] - 2 if df['LoanTerm'][i] - 2 > 0 else 1
        terminal_rate = 0.1
        
        
        results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],warranty,terminal_rate)
        list.append(results['monthlyPayment'])
        
        
        
        total_payment.append(results['monthlyPayment'] * (df['LoanTerm'][i]*12))
        
        
        
        finance_product_profit = results['maintenance_fee'] + results['warranty_fee'] + results['insurance_fee'] + results['business_con_fee'] +  results['travel_labor_cost']
        finance_product.append(finance_product_profit)
        
        
    df['payment'] = list
    df['total_payment'] = total_payment
    df['finance_product_gain'] = finance_product
    df['total_interest'] = df['total_payment'] - df['EquipmentPrice'] - df['finance_product_gain']
    df['total_gain']  = df['total_payment'] - df['EquipmentPrice']

    # create sns line plot of list 
    plt.figure(figsize=(10, 6))
    sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

    plt.axvline(x=df['EquipmentPrice'].iloc[6], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[18], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[38], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[78], color='red', linestyle='--', label='Loan Term = 2 yrs')

    plt.xlabel('Equipment Price')
    plt.ylabel('Monthly Payment')
    # plt.legend()
    # plt.show()
    # plt.savefig('test.png')
    
    return df_random, df['total_gain'].mean(), df
        


In [65]:
df, avg_gain, output_df = run_random_sim()
df

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,9,10,10,10,5
1,2,20000,50000,2,9,8,7,3,5
2,3,50000,100000,5,9,8,5,3,3
3,4,10000,200000,5,9,6,5,3,1
4,5,200000,1000000,5,1,2,5,1,1


In [95]:
# Monthly payment constraint DataFrame
monthly_payment_constraint = pd.DataFrame({
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [5000, 20000, 50000, 100000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Lowest Payment': [300, 500, 750, 1500, 2000],
    'Highest Payment': [1500, 2500, 4000, 6000, 8000]
})

# List to store the top 5 best results
top_results = []

# Initialize counters for infeasible conditions
num_infeasible_too_low = 0
num_infeasible_too_high = 0

# Define a function to update the top 5 results
def update_top_results(avg_gain, df, config, top_results):
    # If the top_results list has less than 5 entries, add the new result
    if len(top_results) < 5:
        top_results.append((avg_gain, df, config))
    else:
        # Check if the new avg_gain is better than the worst in the top 5
        min_avg_gain = min(top_results, key=lambda x: x[0])[0]
        if avg_gain > min_avg_gain:
            # Replace the worst result with the new one
            min_index = top_results.index(min(top_results, key=lambda x: x[0]))
            top_results[min_index] = (avg_gain, df, config)

    # Sort the list by avg_gain in descending order
    top_results.sort(key=lambda x: x[0], reverse=True)

for i in range(2000):
    df_random, avg_gain, df = run_random_sim()  # Get the DataFrame from simulation

    # Check for each row if the constraints are met
    all_constraints_satisfied = True
    for idx, row in df.iterrows():
        # Find the corresponding row in monthly_payment_constraint based on EquipmentPrice
        matching_constraint = monthly_payment_constraint[
            (monthly_payment_constraint['Bottom Price'] <= row['EquipmentPrice']) & 
            (monthly_payment_constraint['Upper Price'] > row['EquipmentPrice'])
        ]
        
        # If no matching constraint found, skip this entry
        if matching_constraint.empty:
            all_constraints_satisfied = False
            break
        
        # Extract the lowest and highest payment bounds
        lowest_payment = matching_constraint['Lowest Payment'].values[0]
        highest_payment = matching_constraint['Highest Payment'].values[0]
        
        # Check if the payment is within bounds
        if row['payment'] < lowest_payment:
            all_constraints_satisfied = False
            num_infeasible_too_low += 1  # Increment counter for too low payments
            break
        elif row['payment'] >= highest_payment:
            all_constraints_satisfied = False
            num_infeasible_too_high += 1  # Increment counter for too high payments
            break

    # If all constraints are satisfied and the average gain is good, consider for top results
    if all_constraints_satisfied:
        update_top_results(avg_gain, df, df_random, top_results)

# After the loop, top_results will contain the best 5 configurations sorted by avg_gain
# top_results is a list of tuples: [(avg_gain, best_df, best_config), ...]

# You can access them like this:
for rank, (gain, df_result, config_result) in enumerate(top_results, start=1):
    print(f"Rank {rank}: Average Gain = {gain}")
    # Optionally, you can inspect df_result and config_result


Rank 1: Average Gain = 92526.55356144806
Rank 2: Average Gain = 89348.95738096167
Rank 3: Average Gain = 88587.98266900377
Rank 4: Average Gain = 88587.92016900377
Rank 5: Average Gain = 88581.14933567044


In [96]:
print(num_infeasible_too_low)
print(num_infeasible_too_high)

495
424


In [98]:
# Access the second best result
best_avg_gain = top_results[1][0]   # The second best average gain
best_df = top_results[1][1]         # The second best DataFrame (df)
best_config = top_results[1][2]     # The second best configuration (df_random)

top_results[1][2] 

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,9,10,10,10,7
1,2,20000,50000,2,9,6,10,7,7
2,3,50000,100000,7,5,4,10,5,3
3,4,10000,200000,7,5,4,10,2,3
4,5,200000,1000000,7,3,2,7,1,1


In [104]:
best_df = top_results[3][1]         # The second best DataFrame (df)
best_config = top_results[3][2] 

best_df.to_csv('best_df_4.csv')
best_config.to_csv('best_config_4.csv')

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def run_best_sim():
    # randomize parameter configuration
    df_random = best_df  # Assuming 'best_df' is already defined elsewhere

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    a = 7500

    # Loop to generate data for 120 entries
    for i in range(120):
        equipment_price = a
        response = get_pricing_details_from_df(equipment_price, df_random)
        loan_term = response['Loan Term']
        
        # Append equipment price and loan term
        data.append({
            'Equipment Price': equipment_price,
            'Loan Term': loan_term
        })
        
        # Append the parameters
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con']
        })
        
        # Increment equipment price for the next iteration
        a += 2500

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    # Initialize your Calculator class (assumed to be defined elsewhere)
    clinic = Calculator()

    # Initialize lists for calculated results
    list_payments = []
    total_payments = []
    finance_product_profits = []

    # Iterate over each entry to compute results using the Calculator
    for i in range(120):
        # Set parameters in the Calculator
        clinic.markup_percentage = params[i]['Markup Price'] / 100
        clinic.warranty_rate = params[i]['Warranty'] / 100
        clinic.maintenance_ratio = params[i]['Maintenance'] / 100
        clinic.insurance_rate = params[i]['Insurance'] / 100
        clinic.business_con_rate = params[i]['Business Con'] / 100
        
        # Set warranty value
        warranty = df['Loan Term'][i] - 2 if df['Loan Term'][i] - 2 > 0 else 1
        terminal_rate = 0.1
        
        # Get the monthly payment from the Calculator
        results = clinic.getMonthlyPayment(df['Equipment Price'][i], df['Loan Term'][i], warranty, terminal_rate)
        list_payments.append(results['monthlyPayment'])
        
        # Compute the total payment
        total_payments.append(results['monthlyPayment'] * (df['Loan Term'][i] * 12))
        
        # Compute finance product profit
        finance_product_profit = (results['maintenance_fee'] + results['warranty_fee'] + 
                                  results['insurance_fee'] + results['business_con_fee'] + 
                                  results['travel_labor_cost'])
        finance_product_profits.append(finance_product_profit)
        
    # Add computed results to the DataFrame
    df['Monthly Payment'] = list_payments
    df['Total Payment'] = total_payments
    df['Finance Product Profit'] = finance_product_profits
    
    # Compute total interest and total gain
    df['Total Interest'] = df['Total Payment'] - df['Equipment Price'] - df['Finance Product Profit']
    df['Total Gain'] = df['Total Payment'] - df['Equipment Price']

    # Add the rate columns from params
    df['Markup Percentage'] = [params[i]['Markup Price'] for i in range(120)]
    df['Warranty Rate'] = [params[i]['Warranty'] for i in range(120)]
    df['Maintenance Rate'] = [params[i]['Maintenance'] for i in range(120)]
    df['Insurance Rate'] = [params[i]['Insurance'] for i in range(120)]
    df['Business Continuity Rate'] = [params[i]['Business Con'] for i in range(120)]

    # Select and reorder the required columns for the final DataFrame
    df = df[[
        'Equipment Price', 'Monthly Payment', 'Loan Term', 'Markup Percentage',
        'Warranty Rate', 'Maintenance Rate', 'Insurance Rate',
        'Business Continuity Rate', 'Total Interest', 'Finance Product Profit', 'Total Gain'
    ]]

    # Plot a line chart of the monthly payments
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='Equipment Price', y='Monthly Payment')

    plt.axvline(x=df['Equipment Price'].iloc[6], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['Equipment Price'].iloc[18], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['Equipment Price'].iloc[38], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['Equipment Price'].iloc[78], color='red', linestyle='--', label='Loan Term = 2 yrs')

    plt.xlabel('Equipment Price')
    plt.ylabel('Monthly Payment')
    plt.legend()
    plt.show()
    plt.savefig('test.png')

    # Return the final DataFrame
    return df


In [42]:
run_best_sim()

,Equipment Price,Monthly Payment,Loan Term,Markup Percentage,Warranty Rate,Maintenance Rate,Insurance Rate,Business Continuity Rate,Total Interest,Finance Product Profit,Total Gain
0,7500,206.458358,5,9,10,10,10,7,2510.001458,2377.5,4887.501458
1,10000,266.944477,5,9,10,10,10,7,3346.668611,2670.0,6016.668611
2,12500,327.430596,5,9,10,10,10,7,4183.335764,2962.5,7145.835764
3,15000,387.916715,5,9,10,10,10,7,5020.002916,3255.0,8275.002916
4,17500,448.402834,5,9,10,10,10,7,5856.670069,3547.5,9404.170069
...,...,...,...,...,...,...,...,...,...,...,...
115,295000,5422.199032,7,1,4,5,1,3,142434.718711,18030.0,160464.718711
116,297500,5467.938007,7,1,4,5,1,3,143641.792599,18165.0,161806.792599
117,300000,5513.676982,7,1,4,5,1,3,144848.866486,18300.0,163148.866486
118,302500,5559.415957,7,1,4,5,1,3,146055.940374,18435.0,164490.940374


In [ ]:
monthly_payment_constraint = pd.DataFrame({
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [5000, 20000, 50000, 10000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Lowest Payment': [500, 800, 1000, 2000, 4500],
    'Highest Payment': [1500, 2500, 3000, 6000, 8000]
})

4500
3847.9819628360183
8000
800
576.1250729102896
2500


In [59]:
best_df

,EquipmentPrice,LoanTerm,payment,total_payment,finance_product_gain,total_interest,total_gain
0,7500,2,421.801042,10123.225000,1650.0,973.225000,2623.225000
1,10000,2,554.068056,13297.633334,2000.0,1297.633334,3297.633334
2,12500,2,686.335069,16472.041667,2350.0,1622.041667,3972.041667
3,15000,2,818.602083,19646.450000,2700.0,1946.450000,4646.450000
4,17500,2,950.869097,22820.858334,3050.0,2270.858334,5320.858334
...,...,...,...,...,...,...,...
115,295000,5,6862.445400,411746.724023,18020.0,98726.724023,116746.724023
116,297500,5,6920.389853,415223.391175,18160.0,99563.391175,117723.391175
117,300000,5,6978.334305,418700.058328,18300.0,100400.058328,118700.058328
118,302500,5,7036.278758,422176.725481,18440.0,101236.725481,119676.725481
